## Import CSV files from analysis script to MongoDB

In [118]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
import json

In [119]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

# confirm that our new database was created
db = mongo.portfolios_db

In [120]:
# reviewing collections in our db
print(db.list_collection_names())


['balanced_portfolio_data', 'spy_price_info', 'portfolio_weights', 'vgk_price_info', 'bond_price_info', 'sche_price_info', 'etf_additional_info', 'conservative_portfolio_data', 'growth_portfolio_data', 'etf_return_risk', 'vong_price_info']


In [121]:
# assign each collection to a variable
etf_list = ['bond','spy','vgk','sche','vong']
portfolio_list = ['conservative', 'balanced', 'growth']

portfolio_data = {}
for portfolio in portfolio_list:
    portfolio_data[portfolio] = db[f"{portfolio}_portfolio_data"]
    
price_info = {}
for etf in etf_list:
    price_info[etf] = db[f"{etf}_price_info"]
    
etf_return_risk = db["etf_return_risk"]
etf_additional_info = db['etf_additional_info']
portfolio_weights = db['portfolio_weights']

In [138]:
import requests

url = "https://gayatrijohn3-d498f365-c54e-4381-857d-9f4ac180634e.socketxp.com/api/portfolio_weights/conservative"

try:
    response = requests.get(url)

    # Check if the response status code is 200 (OK)
    if response.status_code == 200:
        data = response.json()  # Parse the JSON response
        print(data)
    else:
        print("Failed to fetch data. Status code:", response.status_code)

except requests.exceptions.RequestException as e:
    print("Request Error:", e)


{'BOND': 0.63, 'SCHE': 0.0, 'SPY': 0.31, 'VGK': 0.02, 'VONG': 0.04, 'portfolio': 'conservative'}


In [94]:
portfolio_weights.find({'portfolio':'grow'})[0]


IndexError: no such item for Cursor instance

In [115]:
portfolio_data['conservative'].find_one()

{'_id': ObjectId('651b41861d2350e5581024cd'),
 'date': '2020-03-03',
 'BOND_value': 630000.0,
 'SPY_value': 310000.0,
 'VGK_value': 20000.0,
 'SCHE_value': 0.0,
 'VONG_value': 40000.0,
 'conservative_portfolio_value': 1000000.0,
 'conservative_portfolio_return': ''}

In [122]:
portfolio_data_obj=[]
portfolio_data_obj = portfolio_data['conservative'].find({"date" :{'$gte': '2024-01-01'}}, {"_id": 0})
portfolio_data = []
for data in portfolio_data_obj:
    portfolio_data.append(data)

portfolio_data

if not portfolio_data:
    print('hello')

hello


In [123]:
etf_return_risk.find_one()

{'_id': ObjectId('651b41190393ea0e4d1ac5bd'),
 'ticker': 'SPY',
 'annualized_return': 13.533748178831294,
 'annualized_risk': 18.838762611335284}

In [124]:
price_info['bond'].find_one()

{'_id': ObjectId('651b40bb2fad203c693c9fdb'),
 'date': '2012-03-02',
 'ticker_symbol': 'BOND',
 'close_price': 69.96632385253906,
 'dividend': 0.0,
 'trading_volume': 331600,
 'daily_return': 0.35015106755096337,
 'dividend_return': 0.0,
 'total_return': 0.35015106755096337}

In [110]:
x = '2020-01-01'
from datetime import datetime
y = datetime.strptime(x, '%Y-%m-%d')
y

datetime.datetime(2020, 1, 1, 0, 0)

In [127]:
etf_additional_info.find_one()

{'_id': ObjectId('651c37af2d3cbbbcddd03506'),
 'ticker': 'BOND',
 'Net Assets': '3.68B',
 'NAV': 87.02,
 'PE Ratio (TTM)': 1.53,
 'Yield': '3.84%',
 'YTD Daily Total Return': '-0.79%',
 'Beta (5Y Monthly)': 1.01,
 'Expense Ratio (net)': '0.56%',
 'Inception Date': '2012-02-29'}

In [137]:
start_date = '2020-01-01'
price_info['bond'].find({"date" :{'$gte':datetime(2023, 1, 1)}}, {"_id": 0})[0]


IndexError: no such item for Cursor instance

## Creating Flask APIs

In [40]:
# import flask
from flask import Flask, jsonify

# creating an app
app = Flask(__name__)

In [46]:
# flask routes
@app.route("/api/portfolio_weights/<portfolio>")
def portfolio_weights(portfolio):
    """Return the ETF allocation for each portfolio"""
    weights = portfolio_weights.find({'portfolio':portfolio})[0]
    return jsonify(weights)


In [ ]:
if __name__ == "__main__":
    app.run(debug=True)